In [1]:
s_box = {"0": "b", "1": "f", "2": "3", "3": "2", "4": "a", "5": "c", "6": "9", "7": "1",
         "8": "6", "9": "7", "a": "8", "b": "0", "c": "e", "d": "5", "e": "d", "f": "4"}
s_inv = {"0": "b", "1": "7", "2": "3", "3": "2", "4": "f", "5": "d", "6": "8", "7": "9",
         "8": "a", "9": "6", "a": "4", "b": "0", "c": "5", "d": "e", "e": "c", "f": "1"}

Rcon = [[0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0, 0x0],
        [0x1, 0x3, 0x1, 0x9, 0x8, 0xa, 0x2, 0xe,
            0x0, 0x3, 0x7, 0x0, 0x7, 0x3, 0x4, 0x4],
        [0xa, 0x4, 0x0, 0x9, 0x3, 0x8, 0x2, 0x2,
            0x2, 0x9, 0x9, 0xf, 0x3, 0x1, 0xd, 0x0],
        [0x0, 0x8, 0x2, 0xe, 0xf, 0xa, 0x9, 0x8,
            0xe, 0xc, 0x4, 0xe, 0x6, 0xc, 0x8, 0x9],
        [0x4, 0x5, 0x2, 0x8, 0x2, 0x1, 0xe, 0x6,
            0x3, 0x8, 0xd, 0x0, 0x1, 0x3, 0x7, 0x7],
        [0xb, 0xe, 0x5, 0x4, 0x6, 0x6, 0xc, 0xf,
            0x3, 0x4, 0xe, 0x9, 0x0, 0xc, 0x6, 0xc],
        [0x7, 0xe, 0xf, 0x8, 0x4, 0xf, 0x7, 0x8,
            0xf, 0xd, 0x9, 0x5, 0x5, 0xc, 0xb, 0x1],
        [0x8, 0x5, 0x8, 0x4, 0x0, 0x8, 0x5, 0x1,
            0xf, 0x1, 0xa, 0xc, 0x4, 0x3, 0xa, 0xa],
        [0xc, 0x8, 0x8, 0x2, 0xd, 0x3, 0x2, 0xf,
            0x2, 0x5, 0x3, 0x2, 0x3, 0xc, 0x5, 0x4],
        [0x6, 0x4, 0xa, 0x5, 0x1, 0x1, 0x9, 0x5,
            0xe, 0x0, 0xe, 0x3, 0x6, 0x1, 0x0, 0xd],
        [0xd, 0x3, 0xb, 0x5, 0xa, 0x3, 0x9, 0x9,
            0xc, 0xa, 0x0, 0xc, 0x2, 0x3, 0x9, 0x9],
        [0xc, 0x0, 0xa, 0xc, 0x2, 0x9, 0xb, 0x7, 0xc, 0x9, 0x7, 0xc, 0x5, 0x0, 0xd, 0xd]]


def format1(string):
    while len(string) < 2:
        string = '0'+string
    return string


def hex_to_list(data):
    ret = []
    for num in data:
        ret.append(int(num, 16))
    return ret


def dic2list(dic):
    temp = []
    for i in range(16):
        temp.append(dic[i][0])
    return temp


def key_as_bin_list(state):
    bin_list = [0 for i in range(4*len(state))]
    i = 0
    for int_val in state:
        # after binary operation it is of form 5-->0b101, so we cut the first 2
        v = bin(int_val)[2:]
        pad_zeros = '0'*(4-len(v))
        v = pad_zeros + v
        # we take every integer value and convert it to 4 bit binary form
        bin_list[i] = int(v[0])
        bin_list[i+1] = int(v[1])
        bin_list[i+2] = int(v[2])
        bin_list[i+3] = int(v[3])
        i = i+4
    return bin_list


def list_to_hex(state):
    res = ""
    for nibble in state:
        temp = hex(nibble)[2:]
        res += temp
    return res


def hex_to_bin(data):
    ret = [0 for i in range(4*len(data))]
    i = 0
    for nibble in data:
        v = bin(nibble)[2:]
        extra_zeros = '0'*(4-len(v))
        v = extra_zeros+v
        ret[i] = int(v[0])
        ret[i+1] = int(v[1])
        ret[i+2] = int(v[2])
        ret[i+3] = int(v[3])
        i = i+4
    return ret


def bin_list_to_int(state):
    # make the long binary bit list to a single string
    bit_string = ''.join([str(i) for i in state])
    int_list = []
    # take every 4 bits and convert it to integer
    for i in range(0, len(bit_string), 4):
        int_list.append(int(bit_string[i:i+4], 2))
    return int_list


def bitwise_xor(v1, v2):
    v3 = []
    for i in range(len(v1)):
        v3.append(v1[i] ^ v2[i])
    return v3


def bit_xor2(s1, s2):
    s3 = ''
    for i in range(len(s1)):
        s3 = s3+hex((int('0x'+s1[i], 16) ^ int('0x'+s2[i], 16))).split('0x')[1]
    return s3


def sbox(state, box):
    ret = []
    for nibble in state:
        temp = hex(nibble)
        temp = temp[2:]
        ret.append(int(box[temp], 16))
    return ret


def m_0(state):
    mid = [0 for i in range(16)]
    mid[0] = state[4] ^ state[8] ^ state[12]
    mid[1] = state[1] ^ state[9] ^ state[13]
    mid[2] = state[2] ^ state[6] ^ state[14]
    mid[3] = state[3] ^ state[7] ^ state[11]
    mid[4] = state[0] ^ state[4] ^ state[8]
    mid[5] = state[5] ^ state[9] ^ state[13]
    mid[6] = state[2] ^ state[10] ^ state[14]
    mid[7] = state[3] ^ state[7] ^ state[15]
    mid[8] = state[0] ^ state[4] ^ state[12]
    mid[9] = state[1] ^ state[5] ^ state[9]
    mid[10] = state[6] ^ state[10] ^ state[14]
    mid[11] = state[3] ^ state[11] ^ state[15]
    mid[12] = state[0] ^ state[8] ^ state[12]
    mid[13] = state[1] ^ state[5] ^ state[13]
    mid[14] = state[2] ^ state[6] ^ state[10]
    mid[15] = state[7] ^ state[11] ^ state[15]
    return mid


def m_1(state):
    mid = [0]*16
    mid[0] = state[0] ^ state[4] ^ state[8]
    mid[1] = state[5] ^ state[9] ^ state[13]
    mid[2] = state[2] ^ state[10] ^ state[14]
    mid[3] = state[3] ^ state[7] ^ state[15]
    mid[4] = state[0] ^ state[4] ^ state[12]
    mid[5] = state[1] ^ state[5] ^ state[9]
    mid[6] = state[6] ^ state[10] ^ state[14]
    mid[7] = state[3] ^ state[11] ^ state[15]
    mid[8] = state[0] ^ state[8] ^ state[12]
    mid[9] = state[1] ^ state[5] ^ state[13]
    mid[10] = state[2] ^ state[6] ^ state[10]
    mid[11] = state[7] ^ state[11] ^ state[15]
    mid[12] = state[4] ^ state[8] ^ state[12]
    mid[13] = state[1] ^ state[9] ^ state[13]
    mid[14] = state[2] ^ state[6] ^ state[14]
    mid[15] = state[3] ^ state[7] ^ state[11]
    return mid


def mprime(state):
    state = hex_to_bin(state)
    ret = [0 for i in range(64)]
    ret[0:16] = m_0(state[0:16])
    ret[16:32] = m_1(state[16:32])
    ret[32:48] = m_1(state[32:48])
    ret[48:64] = m_0(state[48:64])
    ret = bin_list_to_int(ret)
    return ret


def shift_rows(state, inverse):
    ret = [0]*16
    idx = 0
    for nibble in state:
        ret[idx] = nibble
        if not inverse:
            idx = (idx + 13) % 16
        else:
            idx = (idx + 5) % 16
    return ret


def key_expansion(key):
    # converting the original key to an array of bits
    key_bin = key_as_bin_list(key)
    # initialize the key to 192 (128-->192)
    final_key = [0]*192
    for i in range(64):
        # original key 64 bits == new key 64 bits
        final_key[i] = key_bin[i]
        # original key last 64 bits == new key last 64 bits
        final_key[i+128] = key_bin[i+64]
    for i in range(63):
        final_key[65+i] = key_bin[i]
    final_key[64] = key_bin[63]
    final_key[127] = final_key[127] ^ key_bin[0]
    return bin_list_to_int(final_key)


def four_round(plaintext, key):
    state = [0 for i in range(len(plaintext))]  # four rounds Encryption
    for i in range(len(plaintext)):
        state[i] = int(plaintext[i], 16)
    k = [0 for i in range(len(key))]
    for i in range(len(key)):
        k[i] = int(key[i], 16)
    extended = key_expansion(k)
    k0 = extended[0:16]
    k0prime = extended[16:32]
    k1 = extended[32:48]

    state = bitwise_xor(state, k0)
    state = sbox(state, s_box)
    state = mprime(state)
    state = shift_rows(state, inverse=False)
    state = bitwise_xor(bitwise_xor(Rcon[5], k1), state)

    state = sbox(state, s_box)
    state = mprime(state)
    state = sbox(state, s_inv)

    state = bitwise_xor(bitwise_xor(Rcon[6], k1), state)
    state = shift_rows(state, inverse=True)
    state = mprime(state)
    state = sbox(state, s_inv)
    state = bitwise_xor(state, bitwise_xor(k0prime, k1))
    return list_to_hex(state)


def balanced(state):
    res = state[0]  # checking if balanced
    for nibble in range(1, len(state)):
        res = bit_xor2(res, state[nibble])
    if res == '0':
        return True
    return False


def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3


all_ = []
const = []
input1 = []  # defining inputs
input_list = ["0010000000000001", "0010000111000010",
              "1110000000100000", "0010010000000000", "0000000000000000"]
key = "010110000000000000000000010a0b11"
k = [0 for i in range(len(key))]
for i in range(len(key)):
    k[i] = int(key[i], 16)
extended = key_expansion(k)
k0prime = extended[16:32]
k1 = extended[32:48]
print("k1 is")
print(k1)
print("k1 xor k0 is")
print(bitwise_xor(k0prime, k1))
# --------------------------------------------------------------
input3 = []  # creating inputs with all and constant property
for i in range(16):
    all_.append(hex(i).split('0x')[1])
sets = []
sets2 = []
input2 = []
for s in range(5):
    # input 1 with 1 active nibble and input 2 with 4 active nibbles
    for i in range(16):
        input1.append(all_[i]+input_list[s][1:])
        input2.append(all_[i]+input_list[s][1:7]+all_[i]+input_list[s]
                      [8:10]+all_[i]+input_list[s][11:13]+all_[i]+input_list[s][14:16])
    sets.append(input1)
    sets2.append(input2)
    input1 = []
    input2 = []
candidates = []
a = []
b = []
temp = []
mydicl = []
mydic = {}
# --------------------------------------------------------------------

for s in range(5):
    ciphertexts = []
    mydic = {}
    # Encrypting
    for i in range(16):
        ciphertexts.append(four_round(sets[s][i], key))
    for i in range(16):
        mydic[i] = []

    for key_xor in range(16):
        for k in range(16):
            for j in range(16):  # guessing k0 xor k1
                res1 = bit_xor2(ciphertexts[j][k], hex(key_xor).split('0x')[1])
                temp.append(s_box[res1])
            if balanced(temp):
                mydic[k].append(key_xor)
            temp = []
    mydicl.append(mydic)
final = {}

# Finding intersection of 5 sets to remove false positives
for i in range(16):
    templist = mydicl[0][i]
    for j in range(5):
        templist = intersection(templist, mydicl[j][i])
    final[i] = templist
print("Value of k0 xor k1 obtained")
print(final.values())
# ------------------------------------------------------------
mydicl = []  # Recovering k1
for s in range(5):
    ciphertexts = []
    peeledoff = []
    mydic = {}
    for i in range(16):
        mydic[i] = []
    sum_ = '0'
    sum2 = '0'
    for i in range(16):
        ciphertexts.append(four_round(sets2[s][i], key))
    xor = dic2list(final)
    cl = []
    cll = []
    for j in range(16):
        for i in range(len(ciphertexts[j])):
            cl.append(int(ciphertexts[j][i], 16))
        cll.append(cl)
        cl = []
    datal = []
    for i in range(16):             # Peeling last layer

        data = bitwise_xor(cll[i], xor)
        data = sbox(data, s_box)

        data = mprime(data)             # Inverting shiftrow,mixcolumn
        data = shift_rows(data, inverse=False)
        data = bitwise_xor(Rcon[6], data)
        for i in range(len(data)):
            data[i] = hex(data[i]).split('0x')[1]
        datal.append(data)
    temp = []

    for key_nibble in range(16):
        for k in range(16):
            for j in range(16):
                res2 = bit_xor2(datal[j][k], hex(key_nibble).split('0x')[1])
                temp.append(s_box[res2])
            # print(temp)
            if balanced(temp):
                mydic[k].append(key_nibble)
            temp = []
    mydicl.append(mydic)
final2 = {}

for i in range(16):  # Remove false positives using 5 sets
    templist = mydicl[0][i]
    for j in range(5):
        templist = intersection(templist, mydicl[j][i])
    final2[i] = templist
print("k1 recovered is")
print(final2.values())


k1 is
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 10, 0, 11, 1, 1]
k1 xor k0 is
[0, 0, 8, 0, 8, 8, 0, 0, 0, 1, 0, 10, 0, 11, 1, 1]
Value of k0 xor k1 obtained
dict_values([[0], [0], [8], [0], [8], [8], [0], [0], [0], [1], [0], [10], [0], [11], [1], [1]])
k1 recovered is
dict_values([[0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [10], [0], [11], [1], [1]])
